In [2]:
import pandas as pd
import numpy as np
import os
import time
import math
import datetime
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error


Using TensorFlow backend.


In [3]:
corr_loc = '/home/me/Documents/BWA/data/corr_data/'
for i in os.listdir(corr_loc):
    if(".csv" not in i):
        os.remove(corr_loc+i)
        print(i)
    #print(i)

In [4]:
def get_test(sym):
    corr_loc = '/home/me/Documents/BWA/data/bw_data/'
    data = pd.read_csv(corr_loc+sym)
    data['Open time'] = data['Open time']/1000
    data['Open time'] = data['Open time'].astype('datetime64[s]')
    data['Close time'] = data['Close time']/1000
    data['Close time'] = data['Close time'].astype('datetime64[s]')
    #print(data.head())
    return data
def normalize(data):
    norm_close =  data["Close"].divide(data["Close"].iloc[0])
    return norm_close
def norm(data):
    norm = data[:]/data[0]
    return norm
def create_dataset(dataset, look_back=100):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)

def model_strat(df):
    df['shifted_2'].fillna(1, inplace=True)
    df['Position'] = None
    ct = len(df)
    for row in range(len(df)):
        if(df['Norm Close'].iloc[row]>df['shifted_2'].iloc[row]) and (df['Norm Close'].iloc[row-1]<df['shifted_2'].iloc[row-1]):
            df['Position'].iloc[row] = 1
        if(df['Norm Close'].iloc[row]<df['shifted_2'].iloc[row]) and (df['Norm Close'].iloc[row-1]>df['shifted_2'].iloc[row-1]):
            df['Position'].iloc[row] = -1
        if(row%100==0):
            print("backtesting...{}% done".format((row/ct)*100))

    df['Position'].fillna(method='ffill',inplace=True)
    df['Market Return'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Strategy Return'] = df['Market Return'] * df['Position']
    
    return df,df['Strategy Return'].cumsum()    

In [1]:
ci = 0
for sym in os.listdir(corr_loc):
    curr_data = get_test(sym)
    #print(i)
    close_data = normalize(curr_data)
    curr_data = curr_data.filter(['Close time','Close'])
    train_data = pd.read_csv(corr_loc+sym)
    test_data = close_data.values
    #model_csv = "./model_data.csv"
    #print(curr_data.head())
    norm_test = norm(test_data)
    try:
        sym = sym.replace(".csv","")
        curr_data['Close time'] = curr_data['Close time'].astype('datetime64[ns]')
        curr_data.set_index('Close time',inplace=True, drop=True)
        print("====={} start datetime:{}=====".format(sym,curr_data.index[0]))
        train_data = train_data.iloc[:,1:]
        corr_matrix = train_data.values
        print(sym,corr_matrix.shape)
        total_len = 1439*len(train_data.columns)
        data = corr_matrix.reshape(total_len)
        print(data.shape)
        for c,i in enumerate(train_data.columns):
            if(c==9):
                break
            num_cols = len(train_data.columns)
            #print(num_cols)
            Xs = data[c::num_cols]
            #print(Xs.shape)
            look_back = 1
            trainX, trainY = create_dataset(Xs, look_back)
            testX, testY = create_dataset(test_data, look_back)
            pct = (c/10)*100
            print("{}% trained for {}".format(pct,sym))
            # reshape input to be [samples, time steps, features]
            trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
            testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
            model = Sequential()
            model.add(LSTM(100, input_shape=(1, look_back)))
            model.add(Dense(1))
            model.compile(loss='mean_squared_error', optimizer='adam')
            #model.fit(trainX, trainY, epochs=10, batch_size=100, verbose=0)
            
        model.fit(testX[1439:],testY[1439:], epochs=2)
        testPredict = model.predict(testX[:1439])
        model_data = pd.DataFrame(testPredict)
        test_data = pd.DataFrame(test_data)
        model = None
        model_data.loc[-1] = 0
        shifted_2 = model_data.shift(2)
        concat_data = None
        concat_data = all_data[1440:]
        concat_data['norm_close'] = norm(norm_test[1440:])
        concat_data['shifted_2']= shifted_2.values
        concat_data['shifted_2'].fillna(method='bfill', inplace=True)
        concat_data['Norm Close'] = norm(concat_data['Close'])
        data, shft2 = model_strat(concat_data)
        model_dir = './model_outputs/'+sym+".csv"
        concat_data.to_csv(model_dir)
        print("====={} training complete!=====".format(sym))
        #if(ci%5==0):
            #time.sleep()
    except Exception as e:
        print(e)
#print(train_data.head(),"\n", len(train_data))

NameError: name 'os' is not defined